In [6]:
import itertools
from collections import OrderedDict

def recurnonincreasing(length,minval,maxval):
    """
    This is a generator that iterates over nonincreasing integer sequences
    of length 'length', with values bounded between 'minval' and 'maxval'.
    """
    for currval in range(minval,maxval+1,1):
        if length == 1:
            yield [currval]
        else:
            for x in recurnonincreasing(length-1,minval,currval):
                x.insert(0,currval)
                yield x


def testtetra(seq):
    """
    Given a list of numbers in the order (x,y,z, xb,yb,zb), 
    where (x,y,z) are the lengths of 3 edges coming out of a vertex and
    (xb,yb,zb) are the three edges opposite them (in order),
    check whether they are edge lengths of a tetrahedron.
    
    See Remark 5 in section 3 of 
    Wirth and Dreiding, Edge lengths determining tetrahedrons, 
    Elem. Math. 64 (2009) 160-170.
    """
    if len(seq) !=6:
        return False
    else:
        x,y,z,xb,yb,zb = seq
        # 2D Cayley-Menger for face (xb,yb,zb)
        D1S = -(xb+yb+zb)*(xb+yb-zb)*(yb+zb-xb)*(zb+xb-yb)
        if D1S >= 0:
            #print('D1S failed')
            return False
        else:
            x2,y2,z2,xb2,yb2,zb2 = [n*n for n in seq]
            # 3D Cayley-Menger
            DS = (2*x2*xb2*(y2+yb2+z2+zb2-x2-xb2) + 
                2*y2*yb2*(z2+zb2+x2+xb2-y2-yb2) +
                2*z2*zb2*(x2+xb2+y2+yb2-z2-zb2) +
                (x2-xb2)*(x2-xb2)*(z2-zb2)-(x2+xb2)*(y2+yb2)*(z2+zb2))
            if DS <= 0:
                #print('DS failed')
                return False
            else:
                return True

def tetraperm(nonincseq):
    """
    Given a nonincreasing sequence of 6 integers, this generator returns the 
    30 nonisometric orderings suitable for input into testtetra.
    
    This is based on the discussion at the beginning of section 6 of
    Wirth and Dreiding, Edge lengths determining tetrahedrons, 
    Elem. Math. 64 (2009) 160-170).
    """
    longest = nonincseq[0]
    for k in range(5):
        remaining = nonincseq[1:]
        opposite = remaining.pop(k)
        nextlargest = max(remaining)
        remaining.remove(nextlargest)
        for perm in itertools.permutations(remaining):
            yield [longest, nextlargest, perm[0], opposite, perm[1], perm[2]]

In [15]:
# max integer length
m=7

#bigtable = [seq for noninc in recurnonincreasing(6,1,m) 
#            for seq in tetraperm(noninc) if testtetra(seq)]

#tups = [tuple(ent) for ent in bigtable]
#data=list(set(tups))
#orderedlengths = list(OrderedDict.fromkeys(tups))

lengthsonly = [noninc for noninc in recurnonincreasing(6,1,m) 
              if any([testtetra(seq) for seq in tetraperm(noninc)])]

In [16]:
len(lengthsonly)

545

In [17]:
# list of sets of 6 strictly positive integers
# which are the lengths of the edges of at least one tetrahedron
lengthsonly

[[1, 1, 1, 1, 1, 1],
 [2, 2, 2, 1, 1, 1],
 [2, 2, 2, 2, 1, 1],
 [2, 2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2, 2],
 [3, 2, 2, 1, 1, 1],
 [3, 2, 2, 2, 2, 1],
 [3, 2, 2, 2, 2, 2],
 [3, 3, 2, 2, 2, 1],
 [3, 3, 2, 2, 2, 2],
 [3, 3, 3, 1, 1, 1],
 [3, 3, 3, 2, 2, 1],
 [3, 3, 3, 2, 2, 2],
 [3, 3, 3, 3, 1, 1],
 [3, 3, 3, 3, 2, 1],
 [3, 3, 3, 3, 2, 2],
 [3, 3, 3, 3, 3, 1],
 [3, 3, 3, 3, 3, 2],
 [3, 3, 3, 3, 3, 3],
 [4, 2, 2, 1, 1, 1],
 [4, 3, 2, 1, 1, 1],
 [4, 3, 3, 1, 1, 1],
 [4, 3, 3, 2, 2, 1],
 [4, 3, 3, 2, 2, 2],
 [4, 3, 3, 3, 2, 2],
 [4, 3, 3, 3, 3, 1],
 [4, 3, 3, 3, 3, 2],
 [4, 3, 3, 3, 3, 3],
 [4, 4, 3, 2, 2, 1],
 [4, 4, 3, 2, 2, 2],
 [4, 4, 3, 3, 2, 1],
 [4, 4, 3, 3, 2, 2],
 [4, 4, 3, 3, 3, 1],
 [4, 4, 3, 3, 3, 2],
 [4, 4, 3, 3, 3, 3],
 [4, 4, 4, 1, 1, 1],
 [4, 4, 4, 2, 2, 1],
 [4, 4, 4, 2, 2, 2],
 [4, 4, 4, 3, 2, 2],
 [4, 4, 4, 3, 3, 1],
 [4, 4, 4, 3, 3, 2],
 [4, 4, 4, 3, 3, 3],
 [4, 4, 4, 4, 1, 1],
 [4, 4, 4, 4, 2, 1],
 [4, 4, 4, 4, 2, 2],
 [4, 4, 4, 4, 3, 1],
 [4, 4, 4, 4, 3, 2],
 [4, 4, 4, 4,

In [22]:
m=25

lengthsonly = [noninc for noninc in recurnonincreasing(6,1,m) 
              if any([testtetra(seq) for seq in tetraperm(noninc)])]

In [23]:
# a table (m, number of distinct integer tetrahedron edge length sets with maximum entry m)
[(m,[x[0] for x in lengthsonly].count(m)) for m in range(1,25+1,1)]

[(1, 1),
 (2, 4),
 (3, 14),
 (4, 33),
 (5, 77),
 (6, 147),
 (7, 269),
 (8, 456),
 (9, 737),
 (10, 1139),
 (11, 1712),
 (12, 2479),
 (13, 3502),
 (14, 4829),
 (15, 6554),
 (16, 8698),
 (17, 11402),
 (18, 14738),
 (19, 18814),
 (20, 23693),
 (21, 29615),
 (22, 36593),
 (23, 44899),
 (24, 54581),
 (25, 65890)]

In [24]:
len(lengthsonly)

330876